In [1]:
import sys
sys.path.append('..')

In [2]:
import sentence_embedding_evaluation_german as seeg
import torch
from typing import List

/Users/uh/projects/paper-293/sentence-embedding-evaluation-german/.venv/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## (1) Instantiate your Embedding model
First, you should load your pretrained embedding.

Here we will generate a random embedding for demonstration purposes.

In [3]:
# generate a random embedding
emb_dim = 512
vocab_sz = 128
emb = torch.randn((vocab_sz, emb_dim), requires_grad=False)
emb = torch.nn.Embedding.from_pretrained(emb)
# assert emb.weight.requires_grad == False

## (2) Specify the preprocessing
The `preprocessor` function converts a sentences as string into embedding vectors of numbers.

Here we will convert the input strings with a nonsensical approach into IDs for the Embedding layer.

In [4]:
def preprocesser(batch: List[str], params: dict=None) -> List[List[float]]:
    """ Specify your embedding or pretrained encoder here
    Paramters:
    ----------
    params : dict
        The params dictionary
    batch : List[str]
        A list of sentence as string
    Returns:
    --------
    List[List[float]]
        A list of embedding vectors
    """
    features = []
    for sent in batch:
        try:
            ids = torch.tensor([ord(c) % 128 for c in sent])
        except:
            print(sent)
        h = emb(ids)
        features.append(h.mean(axis=0))
    features = torch.stack(features, dim=0)
    return features

## (3) Training settings

In [5]:
params = {
    'datafolder': '../datasets',
    'batch_size': 64, 
    'num_epochs': 20,
    # 'early_stopping': True,
    # 'split_ratio': 0.2,  # if early_stopping=True
    # 'patience': 5,  # if early_stopping=True
}

## (4) Specify downstream tasks

In [6]:
# All
# downstream_tasks = [
#     'TOXIC', 'ENGAGE', 'FCLAIM', 'VMWE',
#     'OL19-A', 'OL19-B', 'OL19-C',
#     'OL18-A', 'OL18-B', 
#     'ABSD-1', 'ABSD-2', 'ABSD-3',
#     'MIO-S', 'MIO-O', 'MIO-I', 'MIO-D', 'MIO-F', 'MIO-P', 'MIO-A',
#     'SBCH-L', 'SBCH-S',
#     'LSDC'
# ]

# Group tasks
# downstream_tasks = [
#     'ABSD-2', 'MIO-S', 'SBCH-S',  # Sentiment analysis
#     'ENGAGE', 'MIO-P',  # engaging/personal
#     'FCLAIM', 'MIO-A',  # fact-claim (potential fake news), argumentative, reasoning
#     'TOXIC', 'OL19-A', 'OL19-B', 'OL19-C', 'MIO-O', 'MIO-I',  # toxic
# ]

# Current favorites
downstream_tasks = ['FCLAIM', 'VMWE', 'OL19-C', 'ABSD-2', 'MIO-P', 'SBCH-L', 'LSDC']

## (5) Run experiments

In [7]:
%%time
results = seeg.evaluate(downstream_tasks, preprocesser, **params)

nan
CPU times: user 1min 25s, sys: 9.53 s, total: 1min 34s
Wall time: 1min 48s


## (6) Display results

In [8]:
[(res['task'], res['epochs'], res['train']['num'], res['test']['num']) for res in results]

[('FCLAIM', 20, 3244, 944),
 ('VMWE', 20, 6652, 1447),
 ('OL19-C', 20, 1921, 930),
 ('ABSD-2', 20, 19432, 2555),
 ('MIO-P', 20, 4668, 4668),
 ('SBCH-L', 20, 748, 748),
 ('LSDC', 20, 74140, 8602)]

In [9]:
metric = 'f1'
mode = 'train'

[(res['task'], res[mode][metric]) for res in results]

[('FCLAIM', 0.6667694204685574),
 ('VMWE', 0.8278713168971737),
 ('OL19-C', 0.8662155127537741),
 ('ABSD-2', 0.6797035817208728),
 ('MIO-P', 0.8258354755784063),
 ('SBCH-L', 0.6898395721925134),
 ('LSDC', 0.5188966819530618)]

In [10]:
metric = 'f1'
mode = 'test'

[(res['task'], res[mode][metric]) for res in results]

[('FCLAIM', 0.6694915254237288),
 ('VMWE', 0.8230822391154112),
 ('OL19-C', 0.8559139784946237),
 ('ABSD-2', 0.6536203522504892),
 ('MIO-P', 0.8260497000856898),
 ('SBCH-L', 0.6550802139037433),
 ('LSDC', 0.3207393629388514)]

In [11]:
metric = 'f1-balanced'
mode = 'test'

[(res['task'], res[mode][metric]) for res in results]

[('FCLAIM', 0.41301020408163264),
 ('VMWE', 0.4514783927217589),
 ('OL19-C', 0.4611819235225956),
 ('ABSD-2', 0.2635108481262327),
 ('MIO-P', 0.4523697794462694),
 ('SBCH-L', 0.645333450952717),
 ('LSDC', 0.1382732264468123)]

In [12]:
# class label distribution (training)
for res in results:
    print(res['task'], res['train']['num'])
    print(res['train']['distr-train'])
    print(res['train']['distr-pred'])

FCLAIM 3244
{0: 2141, 1: 1103}
{0: 3208, 1: 36}
VMWE 6652
{1: 1145, 0: 5507}
{0: 6652}
OL19-C 1921
{0: 1664, 1: 257}
{0: 1921}
ABSD-2 19432
{2: 1179, 1: 13208, 0: 5045}
{1: 19432}
MIO-P 4668
{0: 3855, 1: 813}
{0: 4668}
SBCH-L 748
{1: 403, 0: 345}
{1: 481, 0: 267}
LSDC 74140
{12: 15019, 8: 7829, 5: 13506, 1: 5294, 11: 13227, 3: 11002, 2: 5704, 13: 346, 10: 749, 7: 382, 9: 143, 0: 469, 6: 377, 4: 93}
{5: 16766, 11: 18186, 12: 21648, 2: 1955, 3: 15564, 7: 19, 10: 1, 8: 1}


In [13]:
# class label distributions (inference)
for res in results:
    print(res['task'], res['test']['num'])
    print(res['test']['distr-test'])
    print(res['test']['distr-pred'])

FCLAIM 944
{0: 630, 1: 314}
{0: 938, 1: 6}
VMWE 1447
{1: 256, 0: 1191}
{0: 1447}
OL19-C 930
{0: 796, 1: 134}
{0: 930}
ABSD-2 2555
{1: 1670, 0: 780, 2: 105}
{1: 2555}
MIO-P 4668
{1: 812, 0: 3856}
{0: 4668}
SBCH-L 748
{0: 346, 1: 402}
{1: 470, 0: 278}
LSDC 8602
{13: 280, 6: 346, 12: 940, 3: 925, 2: 944, 1: 934, 11: 931, 0: 453, 10: 511, 5: 924, 4: 65, 8: 923, 9: 83, 7: 343}
{5: 1610, 11: 2150, 12: 2431, 3: 2228, 2: 183}


In [14]:
len(res['test']['distr-test'])

14